## Đồ án 2: Image Processing <a class="anchor" id="c2"></a>

### Nội dung đồ án <a class="anchor" id="c21"></a>

Nhắc lại: Trong đồ án 1, bạn đã được giới thiệu rằng ảnh được lưu trữ dưới dạng ma trận các điểm ảnh. Mỗi điểm ảnh có thể là một giá trị (ảnh xám) hoặc một vector (ảnh màu).

Trong đồ án này, bạn được yêu cầu thực hiện các chức năng xử lý ảnh cơ bản sau:
    
1. Thay đổi độ sáng cho ảnh

![img](https://imgur.com/oJ8bTv7.jpg)

2. Thay đổi độ tương phản

![img](https://imgur.com/wl8MSu3.jpg)

3. Lật ảnh (ngang + dọc)

![img](https://imgur.com/MOOvIhN.jpg)

4. Chuyển đổi ảnh RGB thành ảnh xám/sepia

- Ảnh xám

![img](https://imgur.com/XEfRXWE.jpg)

- Ảnh sepia

![img](https://imgur.com/YXUPjHY.jpg)

Keywords: convert RGB to grayscale/sepia

5. Làm mờ + sắc nét ảnh

- Làm mờ

![img](https://imgur.com/wZT4vUa.jpg)

- Làm sắc nét

![img](https://imgur.com/H2Fq4Ne.jpg)

Tham khảo tại [đây](https://en.wikipedia.org/wiki/Kernel_(image_processing))

6. Cắt 1/4 ảnh theo kích thước (cắt ở trung tâm)

![img](https://imgur.com/fXebjfO.jpg)

7. Cắt ảnh theo khung:
- Khung hình tròn

![img](https://imgur.com/DEpimhC.jpg)

- Khung là 2 hình ellip chéo nhau

![img](https://i.imgur.com/fPlYioC.png)

8. Viết hàm process_image để gọi những chức năng xử lý ảnh như trên và hàm main xử lý với các yêu cầu sau:

- Cho phép người dùng nhập vào tên tập tin ảnh mỗi khi hàm main được thực thi.
- Cho phép người dùng lựa chọn chức năng xử lý ảnh (từ 1 đến 7) và hiển thị ảnh kết quả. Nếu có lựa chọn 0 sẽ cho phép lưu file đầu ra tương ứng với từng chức năng. Ví dụ:
    - Đầu vào: `cat.png`
    - Chức năng: Làm mờ
    - Đầu ra: `cat_blur.png`

Trong đồ án này, bạn <font style="color:red">**CHỈ ĐƯỢC PHÉP**</font> sử dụng các thư viện sau: `PIL`, `numpy`, `matplotlib`.

Cụ thể, nếu đề yêu cầu bạn viết ra chức năng đó, thì bạn phải thực sự viết ra chức năng đó chứ không phải gọi hàm có sẵn.

- Các bạn sử dụng `PIL` (`open(), save()` từ `Image`) để đọc và ghi; `Matplotlib` (`imshow()` từ `pyplot`) để hiển thị ảnh.

- Được phép sử dụng thư viện `NumPy` tùy ý.

Lưu ý: Để được **điểm tối đa** cho từng chức năng, thời gian thực thi của mỗi chức năng phải nằm trong khoảng thời gian chấp nhận được. Ví dụ với chức năng làm mờ (phức tạp nhất) có thời gian thực thi trên ảnh với kích thước $512 \times 512$ là dưới 15 giây.

### Quy định bài nộp <a class="anchor" id="c22"></a>

- Thực hiện toàn bộ bài làm trên 1 tập tin Jupyter Notebook (.ipynb)

- Bạn nộp tập tin `MSSV.zip` được nén từ thư mục MSSV chứa các tập tin sau:
    1. Báo cáo toàn bộ bài làm: `MSSV.pdf`
    2. Mã nguồn: `MSSV.ipynb`


- Trong đó, nội dung tập tin báo cáo gồm có:
    - Thông tin cá nhân: Họ và tên, MSSV
    - Liệt kê các chức năng đã hoàn thành
    - Ý tưởng thực hiện, mô tả các hàm chức năng
    - Báo cáo cần có số trang và tài liệu tham khảo

- Tập tin `MSSV.zip` không vượt quá 20MB. Ví dụ minh họa cây thư mục bài nộp sau khi giải nén tập tin `MSSV.zip` như sau:
    ```
    MSSV
    ├── MSSV.pdf
    └── MSSV.ipynb
    ```

### Quy định chấm bài <a class="anchor" id="c23"></a>

Những trường hợp sau đây sẽ bị 0 điểm toàn bộ đồ án:
- Nộp sai quy định
- Không nộp báo cáo/mã nguồn
- Thực thi mã nguồn báo lỗi
- Copy sourcecode vào báo cáo
- Không có phần tài liệu tham khảo và trích dẫn [] cần thiết

<font style="color:red">**LƯU Ý: 
- Các bài làm giống nhau (báo cáo hoặc mã nguồn) sẽ nhận 0 điểm toàn bộ phần thực hành
- Những trường hợp lạm dụng các công cụ AI hỗ trợ mà KHÔNG hiểu bài sẽ bị xem xét như cheating và 0 điểm phần thực hành. Việc kiểm tra hiểu bài sẽ được thực hiện ngẫu nhiên qua việc lên bảng sửa bài hoặc vấn đáp
</font>